In [59]:
import math, heapq
from dataclasses import dataclass

# Nodes for intersections
NODES = ["S","A","B","C","E","D"]  # S = start, D = final destination

# Directed edges with travel time (minutes)
TRAVEL_TIMES = {
    ("S","A"): 4,
    ("S","B"): 6,
    ("A","C"): 5,
    ("B","C"): 2,
    ("B","E"): 4,
    ("C","D"): 6,
    ("C","E"): 3,
    ("E","D"): 2,
    ("A","E"): 8
}

#An adjacency list that acts as a map that shows which nodes are directly reachable from each node based off the graph.
ADJ = {n: [] for n in NODES}
for (u,v) in TRAVEL_TIMES:
    ADJ[u].append(v)

#Dijkstra's Algorithm to finds the shortest driving time from the starting node to all other nodes
def dijkstra(start):

    #Store shortest distance from start "S" to each node
    distance = {node: math.inf for node in NODES}
    #Stores the previous node used to reach each node
    previous = {node: None for node in NODES}
    distance[start] = 0
    queue = [(0, start)]

    #Repeatedly select the closest unexplored node and update shorter paths to its neighbors
    while queue:

        current_distance, current_node = heapq.heappop(queue)
        if current_distance > distance[current_node]:
            continue

        for neighbor in ADJ[current_node]:
            new_distance = current_distance + TRAVEL_TIMES[(current_node, neighbor)]
            if new_distance < distance[neighbor]:
                distance[neighbor] = new_distance
                previous[neighbor] = current_node
                heapq.heappush(queue, (new_distance, neighbor))

    return distance, previous

#Builds and returns the shortest path from the start node to the target by following the stored previous node links
def path(previous, target):
    route = []
    current = target

    while current is not None:
        route.append(current)
        current = previous[current]

    route.reverse()
    return route

#Parking lot information
@dataclass(frozen=True)
class Lot:
    name: str
    node: str
    price: float           #Price in dollars
    walk_min: float        #Walking time in minutes
    capacity: int
    occupied: int

    #Function to check if parking lot has spots available
    @property
    def available(self):
        return self.occupied < self.capacity

#4 parking lots for testing
LOTS = [
    Lot("P1","A", price = 5, walk_min = 15, capacity = 10, occupied = 7),
    Lot("P2","B", price = 3, walk_min = 20, capacity = 10, occupied = 10),
    Lot("P3","C", price = 4, walk_min = 10, capacity = 10, occupied = 2),
    Lot("P4","E", price = 10, walk_min = 5, capacity = 10, occupied = 9),
]

#CONSTANTS
MAX_WALK_MIN = 25 #maximum walking amount of 25 minutes
PRICE_WEIGHT = 2.0 #Every dollar for the parking lot price = 2 minutes

#Function that will return the total cost and 
def total_cost(drive_min, lot):
    if not lot.available: return math.inf
    if lot.walk_min > MAX_WALK_MIN: return math.inf
    return drive_min + lot.walk_min + PRICE_WEIGHT*lot.price


In [60]:
#Runs dijkstra's algorithm starting from node S
dist, prev = dijkstra("S")

#Parking lot table
rows = []
for lot in LOTS:
    drive = dist[lot.node]
    walk = lot.walk_min
    tot  = total_cost(drive, lot)
    pth  = "->".join(path(prev, lot.node))
    rows.append((tot, lot.name, lot.node, pth, drive, walk, lot.price, lot.available))

#Sorts and prints each parking lots calculated information
for tot, name, node, pth, drive, walk, price, avail in sorted(rows):
    if not avail:
        status = "(FULL)"
    elif walk > MAX_WALK_MIN:
        status = "(Too Far)"
    else:
        status = ""

    tot_str = "N/A" if math.isinf(tot) else f"{tot:.2f}"
    print(f"{name:<3} {node:<4} {pth:<20} {drive:>10.2f} {walk:>10.2f} {price:>9.2f} {tot_str:>10} {status}")


P3  C    S->B->C                    8.00      10.00      4.00      26.00 
P1  A    S->A                       4.00      15.00      5.00      29.00 
P4  E    S->B->E                   10.00       5.00     10.00      35.00 
P2  B    S->B                       6.00      20.00      3.00        N/A (FULL)
